## RAG with Anthropic Claude-3-5-sonnet
Author: **Peeyush Sharma**; Feedback: **PSharma3@gmail.com**

This notebooks has a sample workflow of Retrival Augmented Generation (RAG). A series of questions have been asked against 15 total PDF files totaling 10MB in size. See the bottom of this page for model responses and a cross check against the original source documents.

Components covered in this scaled-down implementation include:
- Anthropic 'claude-3-5-sonnet-20241022' as LLM.
- Model 'all-MiniLM-L6-v2' for generating embeddings.
- The documents ingested are coming from  Loan Syndications and Trading Association (LSTA) https://www.lsta.org/. The persona provided to the agent is that of an analyst working at LSTA.
- Chroma DB has been used as the Vector DB. The production implementation can use Zilliz, pgVector, MongoDB, Pinecone etcetera based on latency and cost considerations.

Another sample RAG with a locally deployed and therefore much smaller LLM can be cross checked here: https://github.com/psharma1544/JupyterNotebooks/blob/master/LLMs/rag_ollama.ipynb. The major difference is in the quality of responses between a fairly small local LLM and a large model demonstrated in this notebook.


In [1]:
import os
from uuid import uuid4

import chromadb
import torch
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from util import llm_call

In [2]:
DOC_DIR = "../documents"
DOC_TYPE = "LSTA"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
# LLM_MODEL_NAME = "llama3.2"
CHUNK_SIZE = 4096
CHUNK_OVERLAP = 512
PURGE_VECTOR_DB = True

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
collection_name = "rag_collection_lsta"
chroma_client = chromadb.PersistentClient()

try:
    if PURGE_VECTOR_DB:  # Keep option for hard reset
        chroma_client.delete_collection(name=collection_name)
    chroma_client.get_or_create_collection(collection_name)
except ValueError:
    chroma_client.delete_collection(collection_name)

collection = chroma_client.get_or_create_collection(collection_name)

vector_store = Chroma(
    client=chroma_client,
    collection_name=collection_name,
    embedding_function=embeddings
)

In [24]:
# Split the document into chunks
def insert_document_to_vector_db(doc_path):
    loader = PyPDFLoader(doc_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, separator="\n")
    docs = text_splitter.split_documents(documents=documents)
    uuids = [str(uuid4()) for _ in range(len(docs))]
    # print(len(uuids))
    ids = vector_store.add_documents(documents=docs, ids=uuids)
    return ids

In [25]:
from pypdf.errors import PdfStreamError

dir_path = os.path.join(DOC_DIR, DOC_TYPE)
vector_indices = []
if os.path.exists(dir_path) and os.path.isdir(dir_path):
    temp_indices = [insert_document_to_vector_db(os.path.join(dir_path, doc))
                    for doc in os.listdir(dir_path)
                    if doc != '.DS_Store'
                    ]
                    # if doc in file_list]
    vector_indices.extend(temp_indices)
# vector_indices

In [22]:
questions = [
    "What are the core components of Green Loan Principles?",
    "Can the sustainability strategy of a parent impact a borrower’s eligibility for an SLL?",
    "What is an 'Excluded Account' in the context of a credit agreement?",
    "When can a revolving loan be recognized as a social loan?",
    "What are some of the main challenges in application of Sustainability Linked Loan Principles (SLLP) to fund finance facilities?",
    "What is Principal Adverse Impact (PAI)?",
    "What does a Second Party Opinion entail?",
    "Provide a summary of LSTA GUIDANCE REGARDING U.S. SANCTIONS ISSUES IN LENDING TRANSACTIONS for different countries. Cover every country for which a specific recommendation has been made. Ensure that the summary is minimally 1000 words.?",
]

for question in questions:
    print("\n\n----------------------")
    print("Question: " + question)
    retriever = vector_store.as_retriever(search_type="mmr",
                                          search_kwargs={'k': 20})
    context = retriever.invoke(question)

    prompt = f"""
        You are a consultant working for Loan Syndications and Trading Association (LSTA).
        Your have been tasked with helping LSTA answer the following query:
        <query>
        {question}
        </query>
        You have access to the following LSTA documents which are meant to provide context as you answer the query:
        <documents>
        {context}
        </documents>
        You must provide document name and section heading and number from which the answer was generated. Please remain faithful
        to the underlying context, and only deviate from it if you are 100% sure that you know the answer already.
        Answer the question now, and avoid providing preamble such as 'Here is the answer', etc
        """

    result = llm_call(prompt)
    print(f"\nLLM Response: {result}")



----------------------
Question: What are the core components of Green Loan Principles?

LLM Response: According to the "Green Loan Principles" document, under the section "Green Loan Principles – Core Components", there are four core components:

1. Use of Proceeds
2. Process for Project Evaluation and Selection
3. Management of Proceeds
4. Reporting

These core components form the framework that enables all market participants to clearly understand the characteristics of a green loan. The GLP requires that all four components must be aligned for a loan to be considered a green loan, as explicitly stated in the document: "Green loans shall not be considered interchangeable with loans that are not aligned with the four core components of the GLP."


----------------------
Question: Can the sustainability strategy of a parent impact a borrower’s eligibility for an SLL?

LLM Response: Based on "Guidance on Sustainability-Linked Loan Principles 26 March 2025", Section 3.B.I.b:

Yes, the